# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
from engine.types.industry_type import IndustryType


policies = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": 0.0,
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
    "subsidies": {itype.value: 0.0 for itype in IndustryType},
    "minimum_wage": 0.0,
}

In [3]:
from engine.interface.controller import ModelController
controller = ModelController()
model_id = controller.create_model(num_people=100,starting_policies=policies, starting_unemployment_rate=0.0, inflation_rate=0.001)

In [4]:
controller.step_model(model_id,time=10)

INFO:MESA.mesa.model:calling model.step for timestep 1 
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Producing goods...NOT IMPLEMENTED
INFO:root:Producing goods...N

In [5]:
indicators = controller.get_indicators(model_id, start_time=1, end_time=0)
print(indicators)

    Week  Unemployment  GDP  IncomePerCapita  MedianIncome  HooverIndex  \
1      1           1.0    0        44.568402     48.166851            0   
2      2           1.0    0        45.568402     49.166851            0   
3      3           1.0    0        46.568402     50.166851            0   
4      4           1.0    0        47.568402     51.166851            0   
5      5           1.0    0        48.568402     52.166851            0   
6      6           1.0    0        49.568402     53.166851            0   
7      7           1.0    0        50.568402     54.166851            0   
8      8           1.0    0        51.568402     55.166851            0   
9      9           1.0    0        52.568402     56.166851            0   
10    10           1.0    0        53.568402     57.166851            0   

    LorenzCurve  
1             0  
2             0  
3             0  
4             0  
5             0  
6             0  
7             0  
8             0  
9           